# Estimating Loss with Copia

The data for this first part of the tutorial is sourced from the study by Kestemont & Karsdorp (2019), which marked our initial exploration into the realm of unseen medieval literature using statistical methods. The study addresses a key challenge in historical literature research: the incomplete transmission of sources over centuries, leading to a potential underestimation of the literary diversity of the past. By employing eco-diversity statistical methods on Middle Dutch chivalric epics, the study suggests that previous estimates of text loss might have been too conservative. These methods unveil that up to half, or possibly more, of the original texts may have been lost over time, highlighting the need for quantitative tools to adjust our understanding of historical literary diversity.

## Loading the Data

We will utilize the Pandas library to load our dataset for analysis. The following section provides the necessary code and instructions to successfully load and prepare the data for our exploration using Copia.

In [1]:
import pandas as pd

df = pd.read_csv("../datasets/dum.csv", sep=';')
df.columns = 'work', 'signature'
df.head() 

,work,signature
0,Aiol (2),"Breda, Arch. Begijnhof, z.s."
1,Alexanders geesten,"München, Bayerische Staatsbibliotheek, Cod. ge..."
2,Alexanders geesten,"Gent, UB, 2749,6"
3,Alexanders geesten,"Leiden, UB, Ltk. 1196"
4,Alexanders geesten,"Donaueschingen, Fürstl. Fürstenb. Hofbibl., 173"


The Chao1 estimator formula necessitates the provision of two key parameters from our collection: the number of items occurring once ($f_1$) and the number of items occurring twice ($f_2$). These parameters are pivotal for estimating the unseen diversity in our dataset. Copia comes with a utility function designed to streamline the conversion of our list of works into abundance or incidence data, aligning with the requirements of the different estimators (see the tutorial on loading the data for more details).

In [2]:
from copia.data import to_copia_dataset

ds = to_copia_dataset(
    df, data_type="abundance", input_type="observations", index_column="work")
ds

AbundanceData(S_obs=74, f1=44, f2=13, n=166, counts=array([17, 10, 10,  7,  6,  5,  5,  5,  5,  4,  4,  3,  3,  3,  3,  3,  3,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1]))

## Species Richness

We utilize Copia's `copia.estimators` module to estimate unbiased diversity using one of the available estimators. All estimators can be accessed through a single entry point, the `diversity()` function:

In [3]:
from copia.estimators import diversity

round(diversity(ds, method='chao1'))

148

Copia includes a variety of estimators for your convenience:

In [4]:
for method in ('chao1', 'ichao1', 'ace', 'jackknife', 'egghe_proot'):
    print(method, '->', round(diversity(ds, method=method)))

chao1 -> 148
ichao1 -> 166
ace -> 148
jackknife -> 173
egghe_proot -> 165


## Bootstrapped Estimation

The Chao1 estimator provides a lower bound estimate of the unseen species count. However, this estimate itself has an associated range, defined by a lower and upper bound, which represents the confidence interval around the estimate. Two common approaches to obtaining this confidence interval include analytical solutions or bootstrap procedures. In Copia, the bootstrap procedure is implemented to derive these bounds. To enable this feature, set the CI parameter to True when calling the function.

In [5]:
D = diversity(ds, method='chao1', CI=True, n_iter=100, n_jobs=1)
D

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 78.38it/s]


est    148.012975
lci    102.645897
uci    225.651810
Name: 0, dtype: float64